In [4]:
%load_ext autoreload
%autoreload 2

import radon
import torch

_avail_gpus = torch.cuda.device_count()
GPU_ID = 2  # Default GPU ID
if _avail_gpus == 0:
    print("No GPUs available -- will need one for cuda")
    device = "cpu"
else:
    print(f"Available GPUs: {_avail_gpus}")
    print(f"Using GPU ID {GPU_ID} : {torch.cuda.get_device_name(GPU_ID)}")
    torch.cuda.set_device(GPU_ID)
    device = f"cuda:{GPU_ID}"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Available GPUs: 4
Using GPU ID 2 : NVIDIA L40S


In [5]:
a = torch.rand(3, device=GPU_ID)
b = torch.rand(3, device=GPU_ID)
c = torch.zeros_like(a, device=GPU_ID)
radon.add_arrays(a, b, c)
print("Result:", c)

Result: Hello from CUDA thread 0: 0.38 + 0.43 = 0.81
Hello from CUDA thread 1: 0.32 + 0.67 = 1.00
Hello from CUDA thread 2: 0.94 + 0.68 = 1.62
tensor([0.8126, 0.9976, 1.6229], device='cuda:2')


In [6]:
r = radon.Radon()
print("Radon object created:", r)
print("Radon version:", r.version)
print("Radon working successfully on device")

Radon object created: <radon.radon.Radon object at 0x7a58e9a58200>
Radon version: 0.1.0
Radon working successfully on device
